In [0]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, Bidirectional, TimeDistributed, Flatten, Merge
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import model_from_json
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import pandas as pd 
import numpy as np
import csv
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [0]:
esol = pd.read_csv('train.csv')
esol.head()

,File ID,SMILE,Energy
0,0.xyz,C[NH+]1CC1C(C)(C)O,-1879.273366
1,1.xyz,COC1CNCC1=O,-1598.701542
2,2.xyz,Nc1cccc(N)c1,-1633.722192
3,3.xyz,O=CC1COCO1,-1256.754945
4,4.xyz,N#CCOCCO,-1317.974723


In [0]:
esol_smiles = esol['SMILE'].tolist()
esol_solubility = esol['Energy'].tolist()

In [0]:
def generate_onehot_encoding(docs,type="encoded_docs"):

    t = Tokenizer(char_level=True)
    t.fit_on_texts(docs)

    if type=="vocab_size":
        vocab_size = len(t.word_index) + 1
        return vocab_size

    encoded_docs = t.texts_to_sequences(docs)#, mode='count')
    return encoded_docs

In [0]:
len_esol_smiles = []
for SMILE in esol_smiles:
    len_esol_smiles += [len(SMILE)]
    
max_len_esol = max(len_esol_smiles)

esol_vocab_size = generate_onehot_encoding(esol_smiles,'vocab_size')
esol_vocab_size, max_len_esol

(24, 34)

In [0]:
esol_encodings = generate_onehot_encoding(esol_smiles)
esol_sequences = sequence.pad_sequences(esol_encodings, maxlen=max_len_esol)

### Get XYZ Data

In [0]:
def get_geometry_vector(file_id):
	fo=open(file_id,'r')
	vector=[]
	data=fo.readlines()
	for i in range(2,len(data)):
		elements=data[i].split()
		if elements[0]=='C':
			vector.extend([1,0,0,0])
		elif elements[0]=='N':
			vector.extend([0,1,0,0])
		elif elements[0]=='O':
			vector.extend([0,0,1,0])
		elif elements[0]=='H':
			vector.extend([0,0,0,1])
		for j in range(1,len(elements)):
			vector.append(float(elements[j]))
	return vector

In [0]:
fo=open('train.csv')
data=csv.reader(fo)
d={}
count=0
maxx=count
for i in data:
	for j in i[1]:
		if j not in d:
			d[j]=count
			count+=1

In [0]:
X_train_xyz=[]
fo.close()
fo=open('train.csv')
data=csv.reader(fo)
num=0
maxlen=0
for i in data:
	if num==0:
		num=1
		continue
	#print(float(i[2]))
# 	Y_train.append(float(i[2]))
	cur=[]
	for j in i[1]:
		val=d[j]
		one_hot=[0 for i in range(count)]
		one_hot[val]=1
		cur.extend(one_hot)
	maxlen=max(maxlen,len(cur))
	X_train_xyz.append(np.array(cur))
    
X_train_xyz = np.array(X_train_xyz)

In [0]:
lengths = []
for x in X_train_xyz:
    lengths += [len(x)]
    
max_len_xyz = max(lengths)

In [0]:
X_train_xyz = sequence.pad_sequences(X_train_xyz, maxlen=max_len_xyz)

### Feeding data to model

In [0]:
X1 = esol_sequences
X2 = X_train_xyz
Y = esol_solubility
vocab = esol_vocab_size

In [0]:
inp = [X1, X2]
X1_train, X1_test, y_train, y_test = train_test_split(inp, Y, random_state=1024)

In [0]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, Y, random_state=1024)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, Y, random_state=1024)

In [0]:
y1_test[74], y2_test[74]

(-1695.6071291160001, -1695.6071291160001)

In [0]:
X_train = [X1_train, X2_train]
X_test = [X1_test, X2_test]

In [0]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper
precision = as_keras_metric(tf.metrics.precision)
recall = as_keras_metric(tf.metrics.recall)
auc = as_keras_metric(tf.metrics.auc)

In [0]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [0]:
comb_x_train = []
for i in range(len(X1_train)):
  comb_x_train.append(np.concatenate([X1_train[i],X2_train[i]]))

In [0]:
comb_x_test = []
for i in range(len(X2_test)):
  comb_x_test.append(np.concatenate([X1_test[i],X2_test[i]]))
comb_x_test = np.array(comb_x_test)

In [0]:
comb_x_train = np.array(comb_x_train)

In [0]:
comb_x_train.shape

(7500, 986)

In [0]:
layers=2
optimizer ="adam"
model_type ="regression"
model = Sequential()
model.add(Embedding(esol_vocab_size,  32, input_length=986))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# if layers==3:
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
# if optimizer == "adam" and lr!= 0.001:
#     print("Setting learning rate to"+str(lr))
#     optimizer = tf.train.AdamOptimizer(lr)

model.add(Dense(1))
model.compile(loss='mse',optimizer=optimizer, metrics=['mape'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 986, 32)           768       
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 986, 32)           3104      
_________________________________________________________________
max_pooling1d_45 (MaxPooling (None, 493, 32)           0         
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 493, 32)           3104      
_________________________________________________________________
max_pooling1d_46 (MaxPooling (None, 246, 32)           0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 7872)              0         
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 7873      
Total para

In [0]:
model.fit(comb_x_train, y_train, shuffle=True, validation_split=0.1,\
                        epochs=100, batch_size=32, verbose=1,callbacks=[early_stop])

Train on 6750 samples, validate on 750 samples
Epoch 1/100
6750/6750 [==============================] - 17s - loss: 487.4488 - mean_absolute_percentage_error: 32.1391 - val_loss: 173.3363 - val_mean_absolute_percentage_error: 12.0992
Epoch 2/100
6750/6750 [==============================] - 16s - loss: 178.1124 - mean_absolute_percentage_error: 12.6140 - val_loss: 147.9578 - val_mean_absolute_percentage_error: 10.4190
Epoch 3/100
6750/6750 [==============================] - 16s - loss: 145.7445 - mean_absolute_percentage_error: 10.4481 - val_loss: 126.8593 - val_mean_absolute_percentage_error: 8.6603
Epoch 4/100
6750/6750 [==============================] - 16s - loss: 123.8246 - mean_absolute_percentage_error: 8.9837 - val_loss: 113.6059 - val_mean_absolute_percentage_error: 7.9395
Epoch 5/100
6750/6750 [==============================] - 16s - loss: 113.9547 - mean_absolute_percentage_error: 8.3015 - val_loss: 117.1775 - val_mean_absolute_percentage_error: 7.8150
Epoch 6/100
6750/6750 [

In [0]:
y_predict = model.predict(comb_x_test).reshape(1,-1)[0]

In [0]:
import math
import sklearn

rmse = math.sqrt(sklearn.metrics.mean_squared_error(y1_test, y_predict))
print(rmse)

30.54566465961889


In [0]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y1_test, y_predict)

19.46679747618908